In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import time
import requests

In [2]:
url = 'https://devtomanager.com/interviews/'

In [3]:
html = requests.get(url).text

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [5]:
EmployeeName = [i.text.strip().split(', ')[0] for i in soup.find_all('h5', class_ = 'card-title')]
EmployeeName

['Aviv Ben-Yosef', 'Anand Safi', 'Shawn Axsom', 'Kevin Doyle', 'Arnab Sen']

In [6]:
Role = [i.text.strip().split(', ')[1].split('at')[0] for i in soup.find_all('h5', class_ = 'card-title')]
Role

['Tech Executive Consultant',
 'Engineering Manager ',
 'Senior Engineering Manager ',
 'CTO ',
 'Technical Delivery Manager ']

In [7]:
Company = [(len(i.text.strip().split('at '))>1 and (i.text.strip().split('at '))[1] or 'null') for i in soup.find_all('h5', class_ = 'card-title')]
Company

['null', 'Mark43', 'Docker', 'patientMpower', 'AKQA']

In [10]:
Interview_quote = [i.text.strip() for i in soup.find_all('p', class_ = 'card-text')[::2]]
Interview_quote

['“I want to set up managers to succeed, no matter how much background they’ve already got.”',
 '“I very well understand where my interests lie, which is being an enabler first and then a creator rather than being a creator/ maker 100%.”',
 '“Your focus (and challenges) grows from your direct reports, to teams, to departments, executives, and then external customers and partnerships.”',
 '“If I can provide enough direction to allow people to focus on the jobs they’ll do better than I ever could, everybody wins.”',
 '“The switch to management can be highly rewarding and provide a level of independence, authority, and interactivity that a pure software engineering role may not offer.”']

In [11]:
Date = [i.text.strip().split('\n ')[0] for i in soup.find_all('p', class_ = 'card-text')[1::2]]
Date = pd.to_datetime(pd.Series(Date))
Date

0   2021-08-16
1   2021-07-19
2   2021-07-05
3   2021-06-21
4   2021-05-10
dtype: datetime64[ns]

In [12]:
data = []
for r in [i.find_all('a') for i in soup.find_all('p', class_ = 'card-text')]:
    data.append([i.text.strip() for i in r])
data = list([x for x in data if x!= []])
data

[['#coaching', '#consulting'],
 ['#coaching', '#frontend', '#full-stack', '#public-safety'],
 ['#backend', '#coaching', '#information-systems', '#web'],
 ['#full-stack', '#health-tech'],
 ['#consulting', '#digital-marketing', '#e-commerce', '#product']]

In [13]:
def quotes():
    EmployeeName = [i.text.strip().split(', ')[0] for i in soup.find_all('h5', class_ = 'card-title')]
    Company = [(len(i.text.strip().split('at '))>1 and (i.text.strip().split('at '))[1] or 'null') for i in soup.find_all('h5', class_ = 'card-title')]
    Interview_quote = [i.text.strip() for i in soup.find_all('p', class_ = 'card-text')[::2]]
    Date = [i.text.strip().split('\n ')[0] for i in soup.find_all('p', class_ = 'card-text')[1::2]]
    Date = pd.to_datetime(pd.Series(Date))
    data = list([])
    for r in [i.find_all('a') for i in soup.find_all('p', class_ = 'card-text')]:
        data.append([i.text.strip() for i in r])
    data = list([x for x in data if x!= []])
    return pd.DataFrame({'Employee Name': EmployeeName, 'Company': Company, 'Interview Quotes': Interview_quote, 'Date': Date
                        , 'Tags': data})

In [14]:
stud = 'https://devtomanager.com/interviews/page/{}/'

In [15]:
from tqdm import tqdm
import time

In [16]:
tabels = []
for i in tqdm(range(1, 6)):
    url1 = stud.format(i)
    html = requests.get(url1).text
    time.sleep(3)
    soup = BeautifulSoup(html, 'html.parser')
    tabels.append(quotes())

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.42s/it]


In [17]:
data = pd.concat(tabels, axis = 0)
data.shape

(25, 5)

In [18]:
data.head()

,Employee Name,Company,Interview Quotes,Date,Tags
0,Aviv Ben-Yosef,null,"“I want to set up managers to succeed, no matt...",2021-08-16,"[#coaching, #consulting]"
1,Anand Safi,Mark43,“I very well understand where my interests lie...,2021-07-19,"[#coaching, #frontend, #full-stack, #public-sa..."
2,Shawn Axsom,Docker,“Your focus (and challenges) grows from your d...,2021-07-05,"[#backend, #coaching, #information-systems, #web]"
3,Kevin Doyle,patientMpower,“If I can provide enough direction to allow pe...,2021-06-21,"[#full-stack, #health-tech]"
4,Arnab Sen,AKQA,“The switch to management can be highly reward...,2021-05-10,"[#consulting, #digital-marketing, #e-commerce,..."


In [19]:
data.to_csv('Employement_Quotes.csv')